In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import cv2
from typing import Tuple, List
import os

In [2]:
print(f'Tensorflow version: {tf.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'OpenCV version: {cv2.__version__}')
!python --version

Tensorflow version: 2.3.1
Pandas version: 1.1.1
NumPy version: 1.18.5
OpenCV version: 4.4.0


Python 3.6.10 :: Anaconda, Inc.


In [3]:
"""
We turn this on to prevent tensorflow from throwing a fit about
things that take longer than the batch training in the model.fit
call (namely, the tensorboard callback can take more time to
execute than the batch training iteration itself).

NOTE: This can be disabled simply by commenting it out. If you
think there is a weird tensorflow issue happening, do that to see
the full tensorflow logs during runtime.
"""
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Utility Functions

In [4]:
def getImageDirs(root: str) -> List[str]:
    imageDirs = []
    for subDirectory, directory, files in os.walk(root):
        for file in files:
            if file[-4:] == ".jpg":
                path = os.path.join(subDirectory, file)         
                imageDirs.append(path)
    return imageDirs

In [5]:
def preprocessImage(img: np.ndarray) -> tf.Tensor:
    return img.astype(np.float32) / 255.0

In [6]:
def makeOneHot(value: int, size: int) -> np.ndarray:
    onehot = np.zeros(size)
    onehot[value] = 1
    return onehot

In [7]:
def balanceData(data: pd.DataFrame) -> pd.DataFrame:
    data = data.groupby("class")
    data = data.apply(lambda x: x.sample(data.size().min()).reset_index(drop=True))
    return data.reset_index(drop=True)

In [8]:
def makeModel(inputShape: Tuple[int]) -> keras.Model:
    """
    Source: https://www.tensorflow.org/guide/keras/functional#a_toy_resnet_model
    
    Note that I tend to prefer the super-explicit (if somewhat verbose) style. 
    This style is technically unnecessary, but it helps with readability.
    """
    inputs = keras.Input(shape=inputShape, name="Input")
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(inputs)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(x)
    block_1_output = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(x)

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(block_1_output)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(x)
    block_2_output = layers.add([x, block_1_output])

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(block_2_output)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(x)
    block_3_output = layers.add([x, block_2_output])

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(block_3_output)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units=2, activation="softmax")(x)

    return keras.Model(inputs=inputs, outputs=outputs, name="Simple_ResNet")

## Global Variables

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="accuracy",
        min_delta=0.01,
        patience=3,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir="logs",
        write_graph=True,
        write_images=True
    )
]

root = os.getcwd() + "\\Data"
imageDirs = getImageDirs(root)

batchSize = 128

## Data Acquisition and Preprocessing

In [10]:
data = pd.DataFrame([
            {
                "image": preprocessImage(cv2.imread(filename)),
                "class": filename.split(sep="\\")[-2]
            }
            for filename in imageDirs
        ])

In [11]:
data.groupby("class")["class"].value_counts()

class  class
busy   busy     8402
free   free     4182
Name: class, dtype: int64

In [12]:
data = balanceData(data)
data.groupby("class")["class"].value_counts()

class  class
busy   busy     4182
free   free     4182
Name: class, dtype: int64

In [13]:
classes = list(data.groupby("class").groups.keys())

In [14]:
data["onehot"] = data["class"].apply(
    func=lambda x: makeOneHot(classes.index(x), len(classes))
)

### Split the data into train and test subsets

In [15]:
train = data.groupby("class").sample(frac=0.8)
train.groupby("class")["class"].value_counts()

class  class
busy   busy     3346
free   free     3346
Name: class, dtype: int64

In [16]:
test = data.drop(train.index).reset_index(drop=True)
test.groupby("class")["class"].value_counts()

class  class
busy   busy     836
free   free     836
Name: class, dtype: int64

In [17]:
train = train.reset_index(drop=True)

In [18]:
trainDataset = tf.data.Dataset.from_tensor_slices(
    (
        np.array(train["image"].values.tolist()),
        np.array(train["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(train),
    reshuffle_each_iteration=True
).batch(batchSize)

In [19]:
testDataset = tf.data.Dataset.from_tensor_slices(
    (
        np.array(test["image"].values.tolist()),
        np.array(test["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(test),
    reshuffle_each_iteration=True
).batch(batchSize)

## Model Definition

In [20]:
model = makeModel(inputShape=data.loc[0, "image"].shape)

In [21]:
model.compile(
    optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [22]:
model.summary()

Model: "Simple_ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 148, 148, 32) 896         Input[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 146, 146, 64) 18496       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 48, 48, 64)   0           conv2d_1[0][0]                   
______________________________________________________________________________________

## Training and Evaluation

In [23]:
model.fit(
    trainDataset,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
53/53 [==============================] - 8s 155ms/step - loss: 0.7032 - accuracy: 0.5475
Epoch 2/100
53/53 [==============================] - 8s 145ms/step - loss: 0.5168 - accuracy: 0.7936
Epoch 3/100
53/53 [==============================] - 8s 144ms/step - loss: 0.4223 - accuracy: 0.8879
Epoch 4/100
53/53 [==============================] - 8s 144ms/step - loss: 0.4157 - accuracy: 0.8960
Epoch 5/100
53/53 [==============================] - 8s 144ms/step - loss: 0.4141 - accuracy: 0.8972
Epoch 6/100
53/53 [==============================] - 8s 143ms/step - loss: 0.4145 - accuracy: 0.8976
Epoch 00006: early stopping


In [24]:
loss, accuracy = model.evaluate(testDataset)

14/14 [==============================] - 1s 46ms/step - loss: 0.4292 - accuracy: 0.8804


In [25]:
model.predict(testDataset)

array([[2.9673001e-03, 9.9703264e-01],
       [9.9999356e-01, 6.4342744e-06],
       [1.0000000e+00, 7.9006970e-09],
       ...,
       [9.9319577e-01, 6.8041920e-03],
       [9.9998617e-01, 1.3826858e-05],
       [1.0000000e+00, 1.7378110e-10]], dtype=float32)

## Results

In [26]:
print(f'Test Loss: {loss}\nTest Accuracy: {accuracy}')

Test Loss: 0.42916202545166016
Test Accuracy: 0.880382776260376
